In [1]:
!pip install -U spacy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 30.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.5.0 which is incompatible.


In [2]:
!python -m spacy info

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-26 10:51:37.622239: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 10:51:39.308760: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 10:51:39.308901: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [3]:
import json

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

Mounted at /content/drive


In [4]:
csv_path = '/content/drive/MyDrive/23SS_NLP/perfume_info.csv'
file_path = '/content/drive/MyDrive/23SS_NLP/annot_try2.json'
file_path_val = '/content/drive/MyDrive/23SS_NLP/annot_val.json'
txt_path = '/content/drive/MyDrive/23SS_NLP/text_for_labeling_fragrance.txt'

data = pd.read_csv(csv_path).iloc[:,1:4]

f = open(file_path, encoding='UTF-8')
annotation_data_train = json.loads(f.read())

d = open(file_path_val, encoding='UTF-8')
annotation_data_val = json.loads(d.read())

In [5]:
data.head()

,review,name,brand
0,skin get strawberry jelly candies. nostalgic p...,Baccarat Rouge 540,Maison Francis Kurkdjian
1,first sprayed boozy warm apple pie lasted seco...,Angels' Share,By Kilian
2,way masculine cant seem sell bottle... anyone ...,Tobacco Vanille,Tom Ford
3,dry smoky sweet woods. feel like time ive smel...,By the Fireplace,Maison Martin Margiela
4,would love smell someone else unfortunately go...,Lost Cherry,Tom Ford


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  144 non-null    object
 1   name    144 non-null    object
 2   brand   144 non-null    object
dtypes: object(3)
memory usage: 3.5+ KB


In [ ]:
data.review[0]

In [8]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [9]:
for text, annot in tqdm(annotation_data_train['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label = label, alignment_mode = "contract")
    if span is None:
      print("skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

  db.to_disk("./annotation_data_train.spacy")

100%|██████████| 74/74 [00:00<00:00, 182.28it/s]


In [10]:
for text, annot in tqdm(annotation_data_val['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label = label, alignment_mode = "contract")
    if span is None:
      print("skipping entity")
    else:
      ents.append(span)
  doc.ents = ents
  db.add(doc)

  db.to_disk("./annotation_data_val.spacy")

100%|██████████| 29/29 [00:00<00:00, 83.93it/s]


In [11]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-26 10:52:22.571409: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 10:52:22.571577: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 10:52:22.571601: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [12]:
! python -m spacy train config.cfg --output ./ --paths.train ./annotation_data_train.spacy --paths.dev ./annotation_data_val.spacy


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-26 10:52:29.865883: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 10:52:29.866031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 10:52:29.866059: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [13]:
nlp_ner = spacy.load('/content/model-best')


In [14]:
for i in range(0,144):
  globals()[f"doc_{i}"] = nlp_ner(data.review[i]) # dynamic loading

In [15]:
for i in range(0,144):
  globals()[f"doc_{i}_ents"] = list(globals()[f"doc_{i}"].ents) # dynamic loading

In [16]:
doc_1_ents[:5]

[warm, cognac, complex, hint, spicy]

In [17]:
type(doc_0_ents)

list

In [18]:
doc_dict = {}
for i in range(0,144):
  doc_dict[f"doc_{i}"] = globals()[f"doc_{i}_ents"]

In [19]:
# doc마다 향 키워드의 빈도수가 달라서, 빈도수 기준으로 상위 10개 자르고 데이터프레임으로 변환하기
# dictionary로 만들기 이전에 list로 되어있으니까 그때 sorting 하고 dictionary로 만들자
# lemmatize 해야되는데 일단 안하고 돌림.

from collections import Counter
for i in range(0,144):
  globals()[f"counter_{i}"] = Counter(globals()[f"doc_{i}_ents"])
  # globals()[f"doc_{i}_ents_sorted"] = globals()[f"counter_{i}"].most_common(10)


이렇게 하면 제대로 뽑이질 않는거 같은데 다시 뽑아보자. Counter에 문제가 있는것으로 일단 유추했음.

### 원인이 뭘까?

In [20]:
len(doc_0_ents)

588

In [21]:
doc_0_ents[4]

sweet

In [22]:
doc_0_ents[10]

sweet

In [23]:
doc_0_ents[4] == doc_0_ents[10]

False

In [24]:
type(doc_0_ents[0])

spacy.tokens.span.Span

type을 확인해 본 결과 string이 아니라, 자체적인 패키지에서 사용한 spacy.tokens.span.Span 으로 되어있는 것을 확인할 수 있었다.

Spacy 홈페이지에 따르면, 단어간 구조와 의미를 고려하여 DL로 임베딩 output을 도출해내는 형식이기 때문에, 이 token은 학습된 토큰이다. 이런 이유에서 인코딩 디코딩이 사용자 임의대로 원활하지 않은 것으로 보았다.

지금 우리가 하려는 목적은 "학습된" 단어벡터를 얻는것이 아닌, 향 키워드를 추출하기 위함이다. 

따라서 강제적으로 spacy token type을 string으로 바꿔준다.

이후에, 향 키워드가 추출된 이후에 문맥 고려 임베딩을 통해 전체 context를 고려한 임베딩을 실시해준다. 

(ELMo, Word2Vec, Glove, FastText, BERT, 등등...)



### 다시 돌아와서

In [25]:
type(str(1))

str

In [26]:
for i in range(0,144):
  for word in globals()[f"doc_{i}_ents"]:
    globals()[f"doc_{i}_ents_string"] = []
    # string으로 만들어주기
    # append
    globals()[f"doc_{i}_ents_string"].append(str(word))

In [27]:
w = doc_0_ents[0].text
print(type(w))

<class 'str'>


보니까 이렇게는 안되더라. spacy.tokens class를 확인해보려고 들어갔는데

module에 text라는 attribute가 있다더라. 

비슷한 역할을 수행해준다더라.

In [28]:
type(doc_0_ents[0].text)

str

In [29]:
for i in range(0,144):
  for j in range(len(globals()[f"doc_{i}_ents"])):
    globals()[f"text_{i}"] = []
    word = globals()[f"doc_{i}_ents"][j].text
    globals()[f"text_{i}"] = globals()[f"text_{i}"].append(word)

In [30]:
range(len(doc_0_ents))

range(0, 588)

In [31]:
word = doc_0_ents[0].text
word

'strawberry'

In [32]:
type(word)

str

In [33]:
a= [].extend('abs')
a

In [34]:
for j in range(len(doc_0_ents)):
  text_0 = []
  word = doc_0_ents[j].text
  text_0 += word
  j += 1

In [35]:
text_0

['d', 'r', 'y']

In [36]:
text_0

['d', 'r', 'y']

In [37]:
type(text_0)

list

### **해결한 코드** ⭐

In [38]:
tok2text_0 = []
for i in range(len(doc_0_ents)):
  local_word = doc_0_ents[i].text
  tok2text_0 += [local_word]


In [39]:
# 위에 코드를 dynamic 하게 시행할거임.
for i in range(0,144):
  globals()[f"tok2text_{i}"] = []
  for j in range(len(globals()[f"doc_{i}_ents"])):
    local_word = globals()[f"doc_{i}_ents"][j].text
    globals()[f"tok2text_{i}"] += [local_word]


In [40]:
type(tok2text_100[0])

str

해결함!

## 질문: string type을 list에 element로 추가하려고 하는데 append는 왜 잘안되고, 직접 추가하는거는 왜 잘 될까?

이부분 꼭 다시 알아볼것 ⭐⭐⭐

In [41]:
from collections import Counter

for i in range(0,144):
  globals()[f"keyword_{i}"] = [key for key, value in Counter(globals()[f"tok2text_{i}"]).most_common(10)]

In [42]:
keyword_1

['sweet',
 'cognac',
 'warm',
 'vanilla',
 'spicy',
 'unisex',
 'woody',
 'praline',
 'masculine',
 'sweetness']

In [43]:
# make a dataframe consisted of keywords sorted by frequencies of top 10 keywords of fragrances

pd.DataFrame()

""


In [44]:
nameandbrand = data.iloc[:,1:3]
nameandbrand

,name,brand
0,Baccarat Rouge 540,Maison Francis Kurkdjian
1,Angels' Share,By Kilian
2,Tobacco Vanille,Tom Ford
3,By the Fireplace,Maison Martin Margiela
4,Lost Cherry,Tom Ford
...,...,...
139,Interlude Black Iris,Amouage
140,Feminité du Bois,Serge Lutens
141,Halfeti,Penhaligon's
142,Fève Délicieuse,Dior


In [45]:
df = pd.DataFrame.transpose(nameandbrand)
df

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
name,Baccarat Rouge 540,Angels' Share,Tobacco Vanille,By the Fireplace,Lost Cherry,Layton,Ombré Leather (2018),Oud Wood,XJ 1861 Naxos,Cedrat Boise,...,Vanille Fatale,Layton Exclusif,Un Jardin En Mediterranee,La Capitale,Marfa,Interlude Black Iris,Feminité du Bois,Halfeti,Fève Délicieuse,Utopia Vanilla Coco 21
brand,Maison Francis Kurkdjian,By Kilian,Tom Ford,Maison Martin Margiela,Tom Ford,Parfums de Marly,Tom Ford,Tom Ford,Xerjoff,Mancera,...,Tom Ford,Parfums de Marly,Hermès,Xerjoff,Memo Paris,Amouage,Serge Lutens,Penhaligon's,Dior,Kayali Fragrances


In [46]:
df_dict = {}
for i in range(0,144):
  df_dict[f"{i}"] = globals()[f"keyword_{i}"]

In [47]:
df_dict['0']

['sweet',
 'feminine',
 'woody',
 'unisex',
 'warm',
 'sweetness',
 'cloud',
 'floral',
 'masculine',
 'light']

In [48]:
type(df.columns[0])

int

In [49]:
df

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
name,Baccarat Rouge 540,Angels' Share,Tobacco Vanille,By the Fireplace,Lost Cherry,Layton,Ombré Leather (2018),Oud Wood,XJ 1861 Naxos,Cedrat Boise,...,Vanille Fatale,Layton Exclusif,Un Jardin En Mediterranee,La Capitale,Marfa,Interlude Black Iris,Feminité du Bois,Halfeti,Fève Délicieuse,Utopia Vanilla Coco 21
brand,Maison Francis Kurkdjian,By Kilian,Tom Ford,Maison Martin Margiela,Tom Ford,Parfums de Marly,Tom Ford,Tom Ford,Xerjoff,Mancera,...,Tom Ford,Parfums de Marly,Hermès,Xerjoff,Memo Paris,Amouage,Serge Lutens,Penhaligon's,Dior,Kayali Fragrances


In [50]:
df_keyword = pd.DataFrame(df_dict)

In [51]:
df_keyword

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,sweet,sweet,tobacco,vanilla,cherry,vanilla,sweet,wood,tobacco,woody,...,fig,summer,strawberry,vanilla,tuberose,incense,cedar,woody,sweet,vanilla
1,feminine,cognac,vanilla,sweet,sweet,sweet,floral,sweet,honey,citrus,...,green,marine,sweet,sweet,marfa,dark,woody,spicy,vanilla,sweet
2,woody,warm,sweet,smoky,cherry.,spicy,jasmine,woody,sweet,fresh,...,fresh,woody,vanilla,dark,sweet,sweet,sweet,unisex,tonka,summer
3,unisex,vanilla,spicy,woody,bitter,warm,masculine,masculine,lavender,wood,...,citrus,fresh,feminine,woody,green,sweetness,spicy,masculine,cherry,creamy
4,warm,spicy,warm,warm,vanilla,woody,dark,fresh,vanilla,creed,...,summer,aquatic,synthetic,wood,vanilla,vanilla,feminine,vanilla,powdery,jasmine
5,sweetness,unisex,tobacco.,wood,warm,floral,patchouli,spicy,fresh,sweet,...,sweet,sweet,incense,spicy,creamy,smoky,wood,sweet,warm,white floral
6,cloud,woody,vanilla.,sweetness,sweetness,fresh,unisex,money,citrus,vanilla,...,juniper,powdery,super,mature,floral,masculine,warm,halfeti,powder,floral
7,floral,praline,sweetness,cozy,feminine,lavender,warm,vanilla,tonka,synthetic,...,floral,wood,warm,masculine,white,wood,powdery,complex,spicy,tuberose
8,masculine,masculine,unisex,vanilla.,dark,vanilla.,leathery,soft,unisex,magnetic,...,woody,green,sweet.,patchouli,fresh,complex,soft,wood,tonka bean,white
9,light,sweetness,vanille,masculine,patchouli,wood,feminine,tobacco,warm,intense,...,cedar,cedar,elegant,coffee,summer,mysterious,unisex,fresh,lavender,warm


In [52]:
df.shape

(2, 144)

In [53]:
df_keyword.shape

(10, 144)

In [54]:
column_name = list(range(0,144))

In [55]:
df.columns = column_name
df_keyword.columns = column_name

In [56]:
keyword_extracted = pd.concat([df,df_keyword])
keyword_extracted

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
name,Baccarat Rouge 540,Angels' Share,Tobacco Vanille,By the Fireplace,Lost Cherry,Layton,Ombré Leather (2018),Oud Wood,XJ 1861 Naxos,Cedrat Boise,...,Vanille Fatale,Layton Exclusif,Un Jardin En Mediterranee,La Capitale,Marfa,Interlude Black Iris,Feminité du Bois,Halfeti,Fève Délicieuse,Utopia Vanilla Coco 21
brand,Maison Francis Kurkdjian,By Kilian,Tom Ford,Maison Martin Margiela,Tom Ford,Parfums de Marly,Tom Ford,Tom Ford,Xerjoff,Mancera,...,Tom Ford,Parfums de Marly,Hermès,Xerjoff,Memo Paris,Amouage,Serge Lutens,Penhaligon's,Dior,Kayali Fragrances
0,sweet,sweet,tobacco,vanilla,cherry,vanilla,sweet,wood,tobacco,woody,...,fig,summer,strawberry,vanilla,tuberose,incense,cedar,woody,sweet,vanilla
1,feminine,cognac,vanilla,sweet,sweet,sweet,floral,sweet,honey,citrus,...,green,marine,sweet,sweet,marfa,dark,woody,spicy,vanilla,sweet
2,woody,warm,sweet,smoky,cherry.,spicy,jasmine,woody,sweet,fresh,...,fresh,woody,vanilla,dark,sweet,sweet,sweet,unisex,tonka,summer
3,unisex,vanilla,spicy,woody,bitter,warm,masculine,masculine,lavender,wood,...,citrus,fresh,feminine,woody,green,sweetness,spicy,masculine,cherry,creamy
4,warm,spicy,warm,warm,vanilla,woody,dark,fresh,vanilla,creed,...,summer,aquatic,synthetic,wood,vanilla,vanilla,feminine,vanilla,powdery,jasmine
5,sweetness,unisex,tobacco.,wood,warm,floral,patchouli,spicy,fresh,sweet,...,sweet,sweet,incense,spicy,creamy,smoky,wood,sweet,warm,white floral
6,cloud,woody,vanilla.,sweetness,sweetness,fresh,unisex,money,citrus,vanilla,...,juniper,powdery,super,mature,floral,masculine,warm,halfeti,powder,floral
7,floral,praline,sweetness,cozy,feminine,lavender,warm,vanilla,tonka,synthetic,...,floral,wood,warm,masculine,white,wood,powdery,complex,spicy,tuberose


In [57]:
keyword_extracted.to_csv("keyword_by_NER.csv")